In [31]:
using ReadableRegex
using Plots
currentpath = pwd()

"/home/davibarreira/MEGA/EMAp/PlutoLatexConverter.jl"

In [32]:
module Runner
end

Main.Runner

In [33]:
function extractnotebook(notebook)
    s = read(notebook, String)
    cells = split(s, "# ╔═╡ ");
    # The first cell and the final 3 are not used
    codes, contents, outputtag, celltype = [],[],[],[]
    for cell in cells[2:end-3]
        push!(codes, cell[1:36])
        push!(contents, cell[38:end])
        push!(outputtag, endswith(rstrip(cell),";") ? "hideoutput" : "showoutput")
        push!(celltype, cell[38:42] == "md\"\"\"" ? "markdown" : "code")
    end
    
    # Get order and view type
    r = either("# ╠","# ╟")
    sortedcells = split(cells[end],Regex(r))
    sortedcodes = [cell[4:39] for cell in sortedcells[2:end-2]]
    order = [findfirst(isequal(scode),codes) for scode in  sortedcodes[1:end]]
    view  = [occursin("═",c) ? "showcode" : "hidecode" for c in sortedcells[2:end-2]]
    # Matching running order
    view  = view[order]
    
    notebookdata = Dict(:codes => codes, :cells => cells[2:end-3],
                        :contents => contents, :outputtag=>outputtag,
                        :celltype => celltype,:order=> order,:view=>view)
    return notebookdata
end

extractnotebook (generic function with 1 method)

In [44]:
function collectoutputs(notebookdata, notebookfolder="./")
    runpath = pwd()
    cd(notebookfolder)
    expressions=[]
    outputs = []
    for (i, content) ∈ enumerate(nb[:contents])
        if nb[:celltype][i] == "code"
            if startswith(lstrip(content),"begin") && endswith(rstrip(content),"end")
                ex = :($(Meta.parse(strip(content))))
            else
                ex = :($(Meta.parse("begin\n"*content*"\nend")))
            end
            
            s = string(ex.args[end])
            if contains(s, Regex(either("PlutoUI.LocalResource","LocalResource")))
                imagepath = s[findfirst(Regex(look_for(one_or_more(ANY),after="(",before=")")),s)]
                push!(outputs,("imagepath",imagepath))
            else
                println(nb[:contents][i])
                io = IOBuffer();
                Base.invokelatest(show,IOContext(io, :limit => true),
                    "text/plain", dispatch_output(Runner.eval(ex)));
                celloutput = String(take!(io))
                if celloutput == "nothing"
                    celloutput = ""
                end
                push!(outputs, celloutput)
                push!(expressions, ex)
            end
        end
    end
    cd(runpath)
    return outputs, expressions
end

function dispatch_output(command_eval::Plots.Plot)
    savefig(command_eval,"figure1.png")
    return command_eval
end

function dispatch_output(command_eval)
   return command_eval 
end

dispatch_output (generic function with 2 methods)

In [45]:
example = "./example/examplepluto.jl"
nb = extractnotebook(example)

Dict{Symbol, Vector{T} where T} with 7 entries:
  :codes     => Any["7b4b693d-c955-4c9b-97e8-56d28cf74e18", "bdad72d3-5744-459f…
  :outputtag => Any["showoutput", "showoutput", "showoutput", "showoutput", "hi…
  :celltype  => Any["code", "code", "markdown", "code", "code", "code", "code",…
  :view      => ["showcode", "showcode", "showcode", "showcode", "showcode", "s…
  :order     => [3, 1, 2, 4, 5, 6, 7, 8]
  :contents  => Any["using PlutoUI\n\n", "\n\t begin\n\tusing Plots\n\ty(x) = s…
  :cells     => SubString{String}["7b4b693d-c955-4c9b-97e8-56d28cf74e18\nusing …

In [46]:
cd(currentpath)
outputs,expressions = collectoutputs(nb,"./example/");

using PlutoUI



	 begin
	using Plots
	y(x) = sin(x)
	plot(y,
		color=:blue)
end



A = [10,10,10]


x = rand(10);


x .+ 1


